In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import geo

In [ ]:
data = pd.read_csv('data_bc.csv')
dataBC = data[data['addressRegion'] == 'BC']
dataBC = dataBC.sort_values(by='addressLocality')
dataBC.head()

In [41]:
dataBC = dataBC.filter([
    'streetAddress',
    'addressLocality',
    'addressRegion',
    'postalCode',
    'latitude',
    'longitude',
    'price',
    'property-beds',
    'property-baths',
    'property-sqft',
    'Year Built',
    'Area',
    'Garage',
    'Parking Spaces',
    'Property Type', #Agricultural, Condo, Condo/Townhouse, Duplex, Manufactured Home, Mobile Home, MultiFamily, Single Family, Townhome, Vacant Land
    'Square Footage',
    'Waterfront',
    ])
data_bc_single_family = dataBC[dataBC['Property Type'] == 'Single Family']

In [42]:
metro_vancouver_cities = [
    "Vancouver", "Burnaby", "Richmond", "Surrey", "Coquitlam",
    "North Vancouver", "West Vancouver", "New Westminster",
    "Delta", "Port Coquitlam", "Port Moody", "Langley"
]
data_bc_single_family = data_bc_single_family[data_bc_single_family['addressLocality'].isin(metro_vancouver_cities)]
sample = data_bc_single_family.head()
sample

,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,price,property-beds,property-baths,property-sqft,Year Built,Area,Garage,Parking Spaces,Property Type,Square Footage,Waterfront
7409,"51 N SEA AVENUE, Burnaby, BC V5B 1K4",Burnaby,BC,V5B 1K4,49.284733,-122.979824,2188000.0,5.0,3.0,"2,515",NaN,NaN,NaN,NaN,Single Family,"2,515 SQFT",NaN
12098,"7815 ALLMAN STREET, Burnaby, BC V5E 2A9",Burnaby,BC,V5E 2A9,49.235055,-122.950665,3999000.0,8.0,9.0,"6,399",NaN,NaN,NaN,NaN,Single Family,"6,399 SQFT",NaN
6686,"8274 WEDGEWOOD STREET, Burnaby, BC V3N 1C4",Burnaby,BC,V3N 1C4,49.231928,-122.927040,1868000.0,3.0,1.0,"1,775",NaN,NaN,NaN,NaN,Single Family,"1,775 SQFT",NaN
12124,"6555 DENBIGH AVENUE, Burnaby, BC V5H 3R7",Burnaby,BC,V5H 3R7,49.224338,-122.984424,3990000.0,5.0,4.0,"2,736",NaN,NaN,NaN,NaN,Single Family,"2,736 SQFT",NaN
4441,"7088 HALIFAX STREET, Burnaby, BC V5A 1L8",Burnaby,BC,V5A 1L8,49.267710,-122.956486,2998888.0,6.0,5.0,"4,946",NaN,NaN,NaN,NaN,Single Family,"4,946 SQFT",NaN


In [ ]:
# data_bc_single_family.sort_values(by='Price', ascending=False)
data_bc_single_family.to_csv('data_bc_1.csv')

In [43]:
lat_lon_array = sample[['latitude', 'longitude']].to_numpy()
lat_lon_array

array([[  49.2847328, -122.9798237],
       [  49.2350546, -122.950665 ],
       [  49.2319281, -122.9270398],
       [  49.2243379, -122.9844235],
       [  49.2677095, -122.9564857]])

In [ ]:
# write code here for using geo.haversine() to get individual address distances

The code below is for normalizing data once we have it all cleaned up. We can ignore it for now and come back to it once we have clean data with all our columns

In [ ]:
features = ['Price', 'Bedrooms', 'Bathrooms', 'Acreage', 'Square Footage']
scaler = MinMaxScaler()

data_scaled = sample.copy()
data_scaled[features] = scaler.fit_transform(sample[features])

data_scaled["Price"] = 1 - data_scaled['Price']
data_scaled['Garage'] = data_scaled['Garage'].map({'Yes': 1, 'No': 0})
data_scaled['Parking'] = data_scaled['Parking'].map({'Yes': 1, 'No': 0})
data_scaled.head()

In [ ]:
score_features = [
    'Price',
    'Bedrooms',
    'Bathrooms',
    'Square Footage',
    'Acreage',
    'Garage',
    'Parking'
]

weights = np.array([1/len(score_features)] * len(score_features))
data_scored = data_scaled.copy()
data_scored['Score'] = data_scaled[score_features].dot(weights)
data_scored.sort_values(by='Score', ascending=False, inplace=True)
data_scored